In [1]:
import sys
sys.path.append('..')
from model import Model
import torch
from PIL import Image
import numpy as np
from process_data import (
    train_preprocess,
    val_preprocess,
    get_augmentations,
    get_data_loader,
)

In [2]:
model = Model()
model.load_state_dict(torch.load("../model_0.27.pth"))

class Args:
    def __init__(self):
        self.batch_size = 1
        self.num_workers = 1
        self.data_path = "../data/CityScapes"
        
args = Args()

In [3]:
print(model)

Model(
  (model): DeepLabV3(
    (backbone): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [4]:
# Get dataloaders
train_loader, val_loader = get_data_loader(
    args=args, batch_size=2, num_workers=1, validation_size=0.1
)

In [7]:
# Sample a batch of images
train_batch = next(iter(train_loader))
val_batch = next(iter(val_loader))

In [8]:
# Check properties of both train and val batch
print("Train batch:")
print(train_batch[0].shape)
print(train_batch[1].shape)
print(train_batch[0].dtype)
print(train_batch[1].dtype)

print("Val batch:")
print(val_batch[0].shape)
print(val_batch[1].shape)
print(val_batch[0].dtype)
print(val_batch[1].dtype)

# Also get min max values of the images
print("Train batch min max:")
print(train_batch[0].min())
print(train_batch[0].max())

print("Val batch min max:")
print(val_batch[0].min())
print(val_batch[0].max())


Train batch:
torch.Size([2, 3, 520, 1040])
torch.Size([2, 1, 520, 1040])
torch.float32
torch.uint8
Val batch:
torch.Size([2, 3, 520, 1040])
torch.Size([2, 1, 520, 1040])
torch.float32
torch.uint8
Train batch min max:
tensor(-2.1179)
tensor(2.6400)
Val batch min max:
tensor(-1.9980)
tensor(2.5877)
